In [114]:
import os
import pickle
from tweet import Tweet
from nltk.tokenize import TweetTokenizer

In [38]:
ROOT_PICKLE = '../pickle/'

In [147]:
lexicon_attack = ['falso', 'Falso', 'Falso']
word_support = ['verdad']

In [33]:
with open(os.path.join(ROOT_PICKLE, 'chequeado1186118160287698944.json.pkl'), 'rb') as fd:
    json = pickle.load(fd)

In [99]:
def get_text(full_text, text_range):
    begin = text_range[0]
    return full_text[begin:]   

In [111]:
def set_stance(tweet, stance):
    tweet.stance = stance

In [123]:
tknzr = TweetTokenizer()
def tokenizer(tweet, tknzr):
    return tknzr.tokenize(tweet.text)

In [112]:
def get_replies(json):
    replies = []
    for idx, tweet in json.items():
        if 'in_reply_to_status_id_str' not in tweet:
            # Tweet Root
            pass
        ins = tweet['conversation_id_str']
        created_at = tweet['created_at']
        id_str = tweet['id_str']
        full_text = tweet['full_text']
        text_range = tweet['display_text_range']
        text = get_text(full_text, text_range)
        reply_count = tweet['reply_count']
        otweet = Tweet(created_at=created_at,
                       id_str=id_str,
                       text=text,
                       reply_count=reply_count,c
                       in_reply_to_status_id_str=ins
                      )
        replies.append(otweet)
    return replies

In [138]:
tweets = get_replies(json)

In [139]:
# tokenizo el tweet
for tweet in tweets:
    tweet.text = tokenizer(tweet, tknzr)

In [148]:
for tweet in tweets:
    for token in tweet.text:
        if token in lexicon_attack:
            print("ATACK")
            tweet.stance = 'attack'

ATACK
ATACK
ATACK


In [150]:
for key, value in json.items():
    if key == value['conversation_id_str']:
        print(value)

{'created_at': 'Mon Oct 21 03:13:01 +0000 2019', 'id_str': '1186118160287698944', 'full_text': '[NUEVO] Macri habló sobre la baja en la cantidad de secuestros.\n\nLo que dijo es #VERDADERO 👉 https://t.co/RP5ub4rudY\n\nPor @ManuelTarricone.\n#DebateChequeado #DebatePresidencial2019 #DebateAr2019 https://t.co/WZdWATv8Lp', 'display_text_range': [0, 194], 'entities': {'hashtags': [{'text': 'VERDADERO', 'indices': [80, 90]}, {'text': 'DebateChequeado', 'indices': [140, 156]}, {'text': 'DebatePresidencial2019', 'indices': [157, 180]}, {'text': 'DebateAr2019', 'indices': [181, 194]}], 'user_mentions': [{'screen_name': 'ManuelTarricone', 'name': 'Manuel Tarricone', 'id_str': '601259451', 'indices': [122, 138]}], 'urls': [{'url': 'https://t.co/RP5ub4rudY', 'expanded_url': 'http://bit.ly/2oZdGls', 'display_url': 'bit.ly/2oZdGls', 'indices': [93, 116]}], 'media': [{'id_str': '1186118158584893441', 'indices': [195, 218], 'media_url': 'http://pbs.twimg.com/media/EHXwLlAX0AExetz.jpg', 'media_url_htt